<a href="https://colab.research.google.com/github/honggilgim/python_test_with_colab/blob/main/%EC%A0%84%EB%B3%B5%EB%82%98%EC%9D%B4%EC%98%88%EC%B8%A1_1%EC%B0%A8_%EC%8B%9C%EB%8F%84%2C_%EB%A8%B8%EC%8B%A0%EC%97%90_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow
import os

In [6]:
train = pd.read_csv('/content/train2.csv')

In [4]:
test = pd.read_csv('/content/test2.csv')

In [7]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [11]:
data_train = train.drop(index = [47, 382, 435, 847, 1078], axis = 0) # 전복무게 + 껍질무게 > 전체무게

In [13]:
data_train = data_train.drop(index = [119, 129,179, 224, 290, 324, 346, 368, 374, 418, 430, 544, 599,
                                      637, 692, 765, 794, 807, 817, 856, 922, 987, 989, 1013, 1020, 
                                      1035 ,1041, 1057, 1090, 1110, 1112], axis = 0) # 전체 무게 < (전복무게 + 내장무게 + 껍데기 무게) 인 경우(삭제)

In [14]:
train_X = train.drop('Target', axis=1)
train_y = train.Target

train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])
train_X['foreign body'] = foreign_body
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0


foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [15]:
train_X_MF=train_X[(train_X['Gender_F']==1) | (train_X['Gender_M']==1)]
train_y_MF=train_y[(train_X['Gender_F']==1) | (train_X['Gender_M']==1)]
train_X_I=train_X[train_X['Gender_I']==1]
train_y_I=train_y[train_X['Gender_I']==1]

test_MF=test[(test['Gender_F']==1) | (test['Gender_M']==1)]
test_MF_idx=test_MF.index
test_I=test[test['Gender_I']==1]
test_I_idx=test_I.index

train_X_MF.drop(['Gender_F', 'Gender_I', 'Gender_M'], axis=1, inplace=True)
train_X_I.drop(['Gender_F', 'Gender_I', 'Gender_M'], axis=1, inplace=True)

test_MF.drop(['Gender_F', 'Gender_I', 'Gender_M'], axis=1, inplace=True)
test_I.drop(['Gender_F', 'Gender_I', 'Gender_M'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *


In [17]:
temp=train_X
temp['y']=train_y.values
temp['Gender_MF']=temp['Gender_F']+temp['Gender_M']
print(temp.corr())

                Unnamed: 0    Lenght  Diameter    Height  Whole Weight  \
Unnamed: 0        1.000000  0.005647  0.008098  0.027576      0.020325   
Lenght            0.005647  1.000000  0.987336  0.905323      0.926032   
Diameter          0.008098  0.987336  1.000000  0.912596      0.928018   
Height            0.027576  0.905323  0.912596  1.000000      0.897565   
Whole Weight      0.020325  0.926032  0.928018  0.897565      1.000000   
Shucked Weight    0.011309  0.895539  0.891400  0.840838      0.967998   
Viscra Weight     0.026939  0.902617  0.901182  0.868035      0.966080   
Shell Weight      0.024770  0.897628  0.907122  0.902457      0.952435   
Gender_F          0.027089  0.304922  0.315245  0.293724      0.290356   
Gender_I         -0.050490 -0.525160 -0.543525 -0.538969     -0.534727   
Gender_M          0.023087  0.218055  0.226005  0.242108      0.241209   
foreign body      0.029554  0.588485  0.600747  0.611135      0.690211   
y                 0.064267  0.544288  

In [18]:
for MIF in ['MF', 'I']:
    model = Sequential()
    model.add(Dense(16, input_dim=10, activation='elu'))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(64, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(16, activation='elu'))
    model.add(Dense(1))

In [19]:
 model.compile(loss='mean_absolute_error',
                  optimizer='Nadam',
                  metrics=['mae'])

In [23]:
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

In [24]:
rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [28]:
    MODEL_DIR = './model_{0}/'.format(MIF)
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)

    temp='./model_' + MIF  + '/'
    modelpath= temp + '{epoch:02d}-{val_loss:.4f}.hdf5'

In [29]:
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

In [35]:
if MIF=='MF':
  model.fit(train_X_MF, train_y_MF, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])
  Y_prediction = model.predict(test_MF).flatten()
  submission['Target'][test_MF_idx] = np.round(Y_prediction)